In [2]:
from sympy import *
from scipy.optimize import minimize
from scipy.misc import derivative
from scipy.optimize import fsolve

In [3]:
theta1=Symbol('\\theta_1')
theta2 =Symbol('\\theta_2')
lamb =Symbol('\lambda')
ell =Symbol('\ell')
mu1 =Symbol('\mu_1')
mu2 =Symbol('\mu_2')

In [4]:
def profit(theta1,theta2, mu1, mu2, lamb):
    return ((3*lamb)+ mu2*theta2 - mu1*theta1)**2/(18*lamb)


def t(theta1, theta2, mu1, mu2, lamb):
    return (3*lamb + mu2*theta2 - mu1*theta1)/(6*lamb)


def nb_objective(theta1,theta2, mu1, mu2, lamb, ell):
    term1 = (profit(theta1,theta2, mu1, mu2, lamb) - profit(ell,theta2, mu1, mu2, lamb))
    term2 = ( mu2*theta2 + (mu1*theta1- mu2*theta2)*t(theta1, theta2, mu1, mu2,lamb))
    return -term1*term2


def subgame(theta2, mu1, mu2, lamb, ell):
    """solve each firm 1s optimization holding theta 2 fixed"""
    result = minimize(nb_objective, [1], args=(theta2, mu1, mu2, lamb, ell),
                      method='Nelder-Mead', options={'disp': False})
    return result.x

In [5]:
#analysis of subgame #1 
foc =  diff(nb_objective(theta1,theta2, mu1, mu2, lamb, ell), theta1)
print [re(i) for i in solve(foc.subs({ell:70, lamb:10, mu1:.5, mu2:.5, theta2:19.107}),theta1)]
print subgame(19.107, .5, .5, 10, 70)

[15.6860667702325, 78.0974547083938, 98.5374785213737]
[15.68603516]


In [6]:
#analysis of subgame #2
foc =  diff(nb_objective(theta1,theta2, mu1, mu2, lamb, ell), theta1)
solve(foc.subs({ell:60, lamb:10, mu1:.5, mu2:.5, theta2:60}),theta1)

[24.4157801509648, 110.584219849035, 180.000000000000]

In [7]:
#right subgame where theta1 = ell
foc =  diff(nb_objective(theta1,theta2, mu1, mu2, lamb, ell), theta1)
print [re(i) for i in solve(foc.subs({ell:70, lamb:10, mu1:.5, mu2:.5, theta2:70}),theta1)]
print subgame(70, .5, .5, 10, 70)

[30.2678853485771, 121.444593546745, 193.287521104678]
[30.26787109]


In [32]:
print derivative(subgame,19.107, dx=1e-4, args=(.5, .5, 10, 70))


def my_deriv(theta_init, mu1,mu2, lamb,ell):
    
    dx = 1e-8
    theta1=Symbol('\\theta_1')
    
    y1 = (diff(nb_objective(theta1,theta_init[0] +dx, mu1,mu2, lamb,ell), theta1))
    y1 = re(solve(y1,theta1)[0])
    
    y2 = (diff(nb_objective(theta1, theta_init[0] -dx, mu1,mu2, lamb,ell), theta1))
    y2 = re(solve(y2,theta1)[0])
    
    return (y1-y2)/(2*dx)

print my_deriv([19.107], .5, .5, 10, 70)

[0.48828125]
0.205236716510626


In [ ]:
def my_deriv(theta_init, mu1,mu2, lamb,ell):
    
    dx = 1e-8
    theta1=Symbol('\\theta_1')
    
    y1 = (diff(nb_objective(theta1,theta_init[0] +dx, mu1,mu2, lamb,ell), theta1))
    y1 = re(solve(y1,theta1)[0])
    
    y2 = (diff(nb_objective(theta1, theta_init[0] -dx, mu1,mu2, lamb,ell), theta1))
    y2 = re(solve(y2,theta1)[0])
    
    return (y1-y2)/(2*dx)


In [33]:
def seq_objective(theta1, h, mu1, mu2, lamb, ell):
    term1 = (profit(theta1, h, mu1, mu2, lamb) - profit(ell, h.subs(theta1,ell), mu1, mu2, lamb))
    term2 = ( mu2*h+ (mu1*theta1- mu2*h)*t(theta1, h, mu1, mu2,lamb))
    return term1*term2


def eval_foc(theta_init, mu1, mu2, lamb, ell):
    
    theta1=Symbol('\\theta_1')
    
    #h function, analytic and numeric
    h = Function('h')(theta1)
    h_eval = subgame(theta_init, mu1, mu2, lamb, ell)
    #print h_eval
    
    #remember that h(ell) makes things different
    h_ell = h.subs(theta1,ell)
    ell_eval = subgame(ell, mu1, mu2, lamb, ell)
    #print ell_eval
    
    #diff h, analytic and numeric
    diff_h= diff(h)
    #diff_eval = derivative(subgame,theta_init, dx=1e-4, args=(mu1, mu2, lamb, ell))
    diff_eval = my_deriv(theta_init, mu1,mu2, lamb, ell)
    
    #calc foc analytically, then plug in numeric h
    foc = simplify( diff( seq_objective(theta1, h, mu1, mu2, lamb, ell),theta1))
    foc = foc.subs({theta1:theta_init,  h:h_eval, h_ell:ell_eval, diff_h:diff_eval})
    return foc
    
    
#print eval_foc(19.107, .5, .5, 10, 70)
print fsolve(eval_foc,19.107, args=(.5,.5,10,70))

[19.10697177]


In [48]:
foc =  diff(nb_objective(theta1,theta2, mu1, mu2, lamb, ell), theta1)
print [re(i) for i in solve(foc.subs({ell:70, lamb:10, mu1:.5, mu2:.5, theta2:13.55990558}),theta1)]

print subgame(13.55990558, .5, .5, 10, 70)

[14.6875000370381, 73.3309270854788, 87.6612896174831]
[14.6875]


In [96]:
####first solve this way

#h function, analytic and numeric
h = Function('h')(theta1)
h_eval = subgame(18.78067379, .5,.5,10,70)
    
#remember that h(ell) makes things different
h_ell = h.subs(theta1,ell)
ell_eval = subgame(70, .5,.5,10,70)

obj1 = seq_objective(theta1, h, mu1, mu2, lamb, ell)
print obj1.subs({ell:70, lamb:10, mu1:.5, mu2:.5, theta1:18.78067379, h:h_eval, h_ell:ell_eval})

33.5189010504164


In [54]:
#h function, analytic and numeric
h = Function('h')(theta1)
h_eval = subgame(19.107, .5,.5,10,70)
    
#remember that h(ell) makes things different
h_ell = h.subs(theta1,ell)
ell_eval = subgame(70, .5,.5,10,70)

obj2 = seq_objective(theta1, h, mu1, mu2, lamb, ell)
print obj2.subs({ell:70, lamb:10, mu1:.5, mu2:.5, theta1:19.107, h:h_eval, h_ell:ell_eval})

33.5216108430263
